In [ ]:
import numpy as np
from netCDF4 import Dataset
import gzip
import tempfile
import shutil
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [ ]:
dat_file = r'/Users/humphreys/Repositories/personal_calcs/Climate/ORD21164/DNIdm201301160000002231000101MA.nc.gz'

In [ ]:
tmpfile = tempfile.NamedTemporaryFile()
shutil.copyfileobj(gzip.open(dat_file), tmpfile)
nc = Dataset(tmpfile.name, mode='r')

In [ ]:
nc

In [ ]:
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
time = nc.variables['time'][:]
SIC = nc.variables['SID'][:].squeeze()

In [ ]:
fig=plt.figure(figsize=(8,10))
ax = fig.add_axes([0.05,0.05,0.9,0.85])
# map = Basemap(projection='ortho',lat_0=45,lon_0=40,resolution='l')
map = Basemap(llcrnrlon=np.min(lon)-2,llcrnrlat=np.min(lat)-2,urcrnrlon=np.max(lon)+2,urcrnrlat=np.max(lat)+2,projection='mill',fix_aspect = False)
map.drawcoastlines()
map.drawcountries()
lons,lats= np.meshgrid(lon,lat) # for this dataset, longitude is 0 through 360, so you need to subtract 180 to properly display on map
x,y = map(lons,lats)

temp = map.contourf(x,y,SIC)
cb = map.colorbar(temp,"bottom", size="5%", pad="2%")
plt.title('SIC')
cb.set_label('SIC')

In [ ]:
import scipy
fftSIC=np.fft.fft2(SIC)
cor=np.real(np.fft.ifft2(fftSIC*np.conj(fftSIC)))
cor=cor/np.max(cor)

In [ ]:
plt.subplot(1,1,1)
plt.imshow(np.fft.fftshift(cor))
plt.clim(0,1)
plt.colorbar()